In [3]:
import os, sys
import django
PROJECTPATH = '/home/resakse/web/smrp/smrp'
sys.path.insert(0, PROJECTPATH)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "smrp.settings")
os.chdir(PROJECTPATH)
django.setup()

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

ModuleNotFoundError: No module named 'django_htmx'

In [2]:
from exam.models import Ris,Smrp
Ris.objects.all()

ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

In [48]:
from openpyxl import load_workbook
wb = load_workbook(filename = 'SMRP RIS.xlsx', read_only=True)
#sheet_ranges = wb['range names']
ws = wb.worksheets[0]
def get_maximum_rows(ws,col=0):
    rows = 0
    for max_row, row in enumerate(ws, 1):
        # if not all(cols.value is None for cols in row[0]):
        if row[col].value:
            # print(row[col].value)
            rows += 1
    return rows
max_rows = get_maximum_rows(ws,8)
print(max_rows)
print(ws['B3'].value)
from tqdm import tqdm

461
ABDOMEN


In [49]:
rislist = []
col = 8
max_rows = get_maximum_rows(ws,col)
print(max_rows)
pbar = tqdm(total=max_rows,desc='Creating RIS Data...')
for row in ws.iter_rows(min_row=2,max_col=10,max_row=max_rows,values_only=True):
    if row[9]:
        rislist.append(Ris(
            code=row[8],
            name=row[9]))
        pbar.update(1)
pbar.close()


461


Creating RIS Data...: 100%|█████████▉| 460/461 [00:00<00:00, 5472.58it/s]


In [ ]:
Ris.objects.bulk_create(rislist)

In [74]:
rislist = list(Ris.objects.all().values('id','code'))
result = {d['code']: d for d in rislist}


In [120]:
smrplist = []
risid = None
col = 1
max_rows = get_maximum_rows(ws,col)
print(max_rows)
pbar = tqdm(total=max_rows,desc='Creating RIS Data...')
for row in ws.iter_rows(min_row=3,max_col=10,max_row=max_rows,values_only=True):
    if row[1]:
        # print(row[2])
        if row[2] in result:
            risid = result[row[2]]['id']
        smrplist.append(Smrp(
            name=row[1],
            ris_id=risid,
            jxr='Harris'))
        pbar.update(1)
pbar.close()

873



Creating RIS Data...:   0%|          | 0/873 [00:24<?, ?it/s]

Creating RIS Data...: 100%|█████████▉| 871/873 [00:00<00:00, 5085.38it/s]


In [ ]:
Smrp.objects.bulk_create(smrplist)

In [3]:
Ris.objects.all()

<QuerySet [<Ris: Angio Abdominal Aorta>, <Ris: Angio Arch>, <Ris: Angio Carotids/Vertebral>, <Ris: Angio Cerebral>, <Ris: Angio Coeliac>, <Ris: Angio Dacryocystography>, <Ris: Angio Dialysis Fistulogram>, <Ris: Angio Hepatic>, <Ris: Angio Intercostal & Bronchial>, <Ris: Angio Kidney Allograft>, <Ris: Angio Lower Limbs>, <Ris: Angio Pelvis>, <Ris: Angio Pulmonary>, <Ris: Angio Renal>, <Ris: Angio Spinal>, <Ris: Angio Splenic>, <Ris: Angio TIPPS>, <Ris: Angio Thoracic>, <Ris: Angio Upper Limbs>, <Ris: Angio Visceral>, '...(remaining elements truncated)...']>

In [5]:
Ris.objects.filter(name__contains='CT').update(modaliti='CT')
Ris.objects.filter(name__contains='MR').update(modaliti='MRI')
Ris.objects.filter(name__contains='CMR').update(modaliti='MF')
Ris.objects.filter(name__contains='US').update(modaliti='US')
Ris.objects.filter(name__contains='Hardcopy').update(modaliti='HC')
Ris.objects.filter(name__contains='Digitize').update(modaliti='DI')
Ris.objects.filter(name__contains='Fluoro').update(modaliti='FL')
Ris.objects.filter(name__contains='Digitize').update(modaliti='DI')
Ris.objects.filter(name__contains='Angio').update(modaliti='XA')
Ris.objects.filter(name__contains='C-Arm').update(modaliti='MF')
Ris.objects.filter(name__contains='Stereo').update(modaliti='MG')
Ris.objects.filter(name__contains='Mammo').update(modaliti='MG')


10

In [7]:
Ris.objects.filter(name__contains='MRI').update(modaliti='MR')


50

In [3]:
def get_percent(modaliti):
    data = {}
    obj = Ris.objects.filter(modaliti=modaliti)
    data['total'] = obj.count()
    taksiap = obj.filter(smrp=None).count()
    data['siap'] = obj.count() - taksiap
    return data

In [4]:
modaliti_choices = (
    ('CT','CT-Scan'),
    ('MR','MRI'),
    ('XR','General X-Ray'),
    ('MG','Mammography'),
    ('US','Ultrasound'),
    ('FL','Fluoroscopy'),
    ('XA','Angiography'),
    ('MF','Mobile Fluoro/OT/ERCP'),
    ('HC','Hardcopy'),
    ('DI','Digitize'),
    ('RE','Rad Resources'),
    ('RC','Ref Consult'),
)

In [5]:
progress = []
for k,v in modaliti_choices:
    progress.append({v: get_percent(k)})

In [6]:
ris = Ris.objects.filter(modaliti='CT')

In [7]:
ris.count()

62

In [8]:
ris.filter(smrp=None).count()

49

In [10]:
progress


[{'CT-Scan': {'total': 62, 'siap': 13}},
 {'MRI': {'total': 67, 'siap': 0}},
 {'General X-Ray': {'total': 61, 'siap': 2}},
 {'Mammography': {'total': 10, 'siap': 0}},
 {'Ultrasound': {'total': 40, 'siap': 0}},
 {'Fluoroscopy': {'total': 36, 'siap': 0}},
 {'Angiography': {'total': 145, 'siap': 0}},
 {'Mobile Fluoro/OT/ERCP': {'total': 5, 'siap': 0}},
 {'Hardcopy': {'total': 10, 'siap': 10}},
 {'Digitize': {'total': 11, 'siap': 11}},
 {'Rad Resources': {'total': 2, 'siap': 0}},
 {'Ref Consult': {'total': 10, 'siap': 0}}]

In [7]:
rows = Ris.objects.annotate(mesin=Concat(get_modaliti_display).values_list('code', 'name', 'modaliti', 'smrp')
row_num=2
for row in rows:
    for col_num in range(1,len(row)):
        # ws.cell(row=row_num,column=col_num, value=str(rows[col_num]))
        print(row[col_num])
    row_num += 1
    print(row_num)

SyntaxError: invalid syntax (2119742742.py, line 2)

In [10]:
a = Ris.Ris.objects.annotate(mesin=Concat(get_modaliti_display)).values_list('code', 'name', 'modaliti', 'smrp')


AttributeError: type object 'Ris' has no attribute 'Ris'

In [6]:
risku = Ris.objects.first()
risku.mesin

'Angiography'

In [21]:
from django.db.models import Case, CharField, Value, When

choices = dict(Ris._meta.get_field('modaliti').flatchoices)
whens = [When(modaliti=k, then=Value(v)) for k, v in choices.items()]
survey_counts = (
    list(Ris.objects
    .annotate(get_modaliti_display=Case(*whens, output_field=CharField()))
    .values_list('code', 'name','get_modaliti_display','smrp'))
)


In [1]:
Ris.objects.all()

NameError: name 'Ris' is not defined